In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json
import pickle

import lightgbm as lgb
print("Lightgbm version:", lgb.__version__)

import optuna
print("Optuna version:", optuna.__version__)

# local modules
import sys
sys.path.append("../src")
from preproc import process_train_data

Lightgbm version: 4.5.0
Optuna version: 4.0.0


***
### load and preprocess data

In [2]:
# define some paths
path_raw = Path("../data/raw")
path_processed = Path("../data/processed")
path_results = Path("../data/results")

# load data
df_train = pd.read_csv(path_raw / "train.csv")
df_test = pd.read_csv(path_raw / "test.csv")

df_train

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",2,0,13,-0.733333
233230,233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",9,1,5,0.266667
233231,233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",11,3,1,0.666667
233232,233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",24,2,4,0.666667


In [3]:
df_train, numerical_cols, categorical_cols, encoder, scaler = process_train_data(
    df_train,
    scale=False,
    # numerical_cols=numerical_cols,
    # categorical_cols=categorical_cols,
    include_position_features=False,
    include_text_features=False,
)

# Print the results
print("Numerical Columns:", len(numerical_cols))
print("Categorical Columns:", len(categorical_cols))

number of all nan cols:  18
number of constant cols:  198
Numerical Columns: 588
Categorical Columns: 10


***
### optimization



In [4]:
split_list = pickle.load(open('../data/splits/cv2_Game.pkl', 'rb'))

In [5]:
def train_and_score(numerical_cols, categorical_cols):

    # Define the parameters
    params = {
        'objective': "regression",
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'max_depth': 10,
        'learning_rate': 0.1,
        'reg_alpha': 1e-1,
        'reg_lambda': 1e-1,
        'feature_fraction': 0.8,
        'verbose': -1,
        'seed': 2112,
    }

    target = 'utility_agent1'
    oof_scores = []

    # Perform cross-validation
    for _, (train_index, val_index) in enumerate(split_list, 1):
        # Split the data
        X_train, X_val = df_train.iloc[train_index], df_train.iloc[val_index]
        y_train, y_val = X_train[target], X_val[target]
        
        # Create LightGBM datasets
        train_data = lgb.Dataset(
            data=X_train[numerical_cols + categorical_cols],
            label=y_train,
            categorical_feature=categorical_cols,
            free_raw_data=True
        )
        val_data = lgb.Dataset(
            data=X_val[numerical_cols + categorical_cols],
            label=y_val,
            categorical_feature=categorical_cols,
            free_raw_data=True
        )

        # Train the model
        model = lgb.train(
            params,
            train_data,
            num_boost_round=1000,
            valid_sets=[val_data],
        )

        # predict on validation set
        y_pred = model.predict(X_val[numerical_cols + categorical_cols])
        y_pred = np.clip(y_pred, -1, 1)
        
        # Compute RMSE on scaled values
        rmse = np.sqrt(np.mean((y_pred - y_val) ** 2))
        oof_scores.append(rmse)

    return np.mean(oof_scores)


In [6]:
def objective(trial):
    # Feature selection
    selected_numerical = [col for col in numerical_cols if trial.suggest_int(f'use_{col}', 0, 1) == 1]
    selected_categorical = [col for col in categorical_cols if trial.suggest_int(f'use_{col}', 0, 1) == 1]

    if len(selected_numerical) + len(selected_categorical) == 0:
        return 1.
        
    # Call the train_and_score function with selected features
    score = train_and_score(selected_numerical, selected_categorical)
    return score

In [7]:
do_optimize = False
timeout = 3600 * 72

# Start with QMC for good coverage
study = optuna.create_study(
    study_name="optuna_lgbm",
    direction='minimize',
    storage='sqlite:///optuna_lgbm.db',
    load_if_exists=True,
    sampler=optuna.samplers.QMCSampler()
)

if do_optimize:
    study.optimize(objective, n_trials=1000)

    # Switch to TPE for refined search
    study = optuna.create_study(
        study_name="optuna_lgbm",
        direction='minimize', 
        storage='sqlite:///optuna_lgbm.db',
        load_if_exists=True,
        sampler=optuna.samplers.TPESampler(
            n_startup_trials=1,    # Increase random sampling at start
            n_ei_candidates=100,   # Consider more candidates
            multivariate=True,     # Enable multivariate sampling
            constant_liar=True     # Help with parallel optimization
        )
    )
    study.optimize(objective, n_trials=2000)

/var/folders/jw/hkbjn4190p76_0dtnrpy7zwm0000gn/T/ipykernel_91340/790234442.py:10: ExperimentalWarning: QMCSampler is experimental (supported from v3.0.0). The interface can change in the future.
  sampler=optuna.samplers.QMCSampler()
[I 2024-11-25 08:14:48,561] Using an existing study with name 'optuna_lgbm' instead of creating a new one.


In [8]:
study.trials_dataframe().sort_values("value", ascending=True).head(20)

,number,value,datetime_start,datetime_complete,duration,params_use_Absolute,params_use_AbsoluteDirections,params_use_AddDecision,params_use_AddDecisionFrequency,params_use_AddEffect,...,params_use_agent1_exploration_const,params_use_agent1_playout,params_use_agent1_score_bounds,params_use_agent1_selection,params_use_agent2,params_use_agent2_exploration_const,params_use_agent2_playout,params_use_agent2_score_bounds,params_use_agent2_selection,state
2842,2842,0.421874,2024-11-24 18:25:27.206935,2024-11-24 18:26:39.146808,0 days 00:01:11.939873,0,1,0,0,1,...,0,0,0,1,1,1,1,1,0,COMPLETE
1618,1618,0.422190,2024-11-23 19:10:19.053700,2024-11-23 19:11:22.702021,0 days 00:01:03.648321,0,1,0,1,1,...,1,1,0,1,1,1,1,0,0,COMPLETE
2637,2637,0.423043,2024-11-24 14:20:21.037468,2024-11-24 14:21:32.329225,0 days 00:01:11.291757,0,1,0,1,1,...,0,1,0,0,1,1,0,0,1,COMPLETE
1876,1876,0.423284,2024-11-23 23:50:40.941587,2024-11-23 23:51:46.742809,0 days 00:01:05.801222,0,1,1,0,0,...,0,1,1,1,1,0,1,0,0,COMPLETE
2898,2898,0.423553,2024-11-24 19:32:58.912899,2024-11-24 19:34:12.135577,0 days 00:01:13.222678,0,1,1,1,0,...,0,1,0,0,1,1,0,1,0,COMPLETE
1970,1970,0.423756,2024-11-24 01:34:35.969325,2024-11-24 01:35:41.913440,0 days 00:01:05.944115,0,1,1,0,0,...,0,1,0,1,1,1,1,0,0,COMPLETE
1875,1875,0.423766,2024-11-23 23:49:34.763291,2024-11-23 23:50:40.906194,0 days 00:01:06.142903,0,1,0,0,0,...,0,1,1,1,1,1,1,0,0,COMPLETE
2609,2609,0.423798,2024-11-24 13:47:06.182357,2024-11-24 13:48:16.243395,0 days 00:01:10.061038,0,0,0,0,1,...,0,1,1,1,1,1,1,0,0,COMPLETE
1535,1535,0.423841,2024-11-23 17:41:13.383369,2024-11-23 17:42:18.249103,0 days 00:01:04.865734,0,1,1,0,0,...,0,1,0,0,1,0,0,1,0,COMPLETE
2959,2959,0.424032,2024-11-24 20:47:30.444248,2024-11-24 20:48:44.353187,0 days 00:01:13.908939,0,1,1,0,0,...,0,1,0,0,1,0,0,1,0,COMPLETE


In [9]:
# Get the selected features from the best trial
best_params = study.best_trial.params

# Filter numerical and categorical features
selected_numerical = [col for col in numerical_cols if best_params.get(f'use_{col}', 0) == 1]
selected_categorical = [col for col in categorical_cols if best_params.get(f'use_{col}', 0) == 1]

# Combine selected features
selected_features = selected_numerical + selected_categorical

print("Selected features:")
print("len(numerical): ", len(selected_numerical), "/", len(numerical_cols))
print("len(categorical): ", len(selected_categorical), "/", len(categorical_cols))
print("len(selected_features): ", len(selected_features), "/", len(numerical_cols) + len(categorical_cols))


# Save selected features to a file
selected_features_dict = {
    "numerical": selected_numerical,
    "categorical": selected_categorical
}
with open('select_optuna_lgbm.json', 'w') as f:
    json.dump(selected_features_dict, f, indent=2)

print("Selected features have been saved to 'select_optuna_lgbm.json'")

Selected features:
len(numerical):  287 / 588
len(categorical):  6 / 10
len(selected_features):  293 / 598
Selected features have been saved to 'select_optuna_lgbm.json'


***